In [3]:
from lightning_sdk import Machine, MMT, Studio

In [1]:
from utils.master_node import MasterNodeServer
private_master_host_ip_address = MasterNodeServer.get_master_ip()
public_master_host_ip_address = MasterNodeServer.get_master_public_ip_curl()
public_master_host_ip_address_services = MasterNodeServer.get_master_public_ip()
print(f"private_master_host_ip_address = {private_master_host_ip_address}")
print(f"public_master_host_ip_address = {public_master_host_ip_address}")
print(f"public_master_host_ip_address = {public_master_host_ip_address_services}")

private_master_host_ip_address = 10.192.12.204
public_master_host_ip_address = 3.84.102.51
public_master_host_ip_address = 3.84.102.51


In [2]:
# Configuration
import os
NUM_NODES = 2
NUM_GPUS = 8
TEAMSPACE = "general"  # Replace with your teamspace
USER = "meta-ai"  # Replace with your username
MONARCH_DEFAULT_PORT = 26600 # Monarch default port
HTTP_SERVER_PORT = MONARCH_DEFAULT_PORT # 8080 # HTTP Server PORT for IP registration

os.environ["MONARCH_FILE_LOG"] = "debug"

In [7]:
print(os.environ.get("MONARCH_FILE_LOG"))

debug


In [ ]:
def launch_mmt_job(num_nodes=2, teamspace="my-teamspace", user="my-user"):
    """
    Launch a multi-machine training job using Lightning SDK's MMT API.
    """

    studio = Studio()

    # Install the MMT plugin befor running the actual job
    studio.install_plugin("multi-machine-training")

    print(f"Launching MMT job with {num_nodes} nodes...")

    # Machine with T4 GPUs
    # machine_type = getattr(Machine, f"T4_X_{NUM_GPUS}")

     # Machine with L40 GPUs
    # machine_type = getattr(Machine, f"L4_X_{NUM_GPUS}")

    # Machine with L40S GPUs
    machine_type = getattr(Machine, f"L40S_X_{NUM_GPUS}")

    job = MMT.run(
        command=f"python example/utils/worker_node.py {public_master_host_ip_address} {HTTP_SERVER_PORT} && sleep 10 && process_allocator",
        name=f"Multi-Node-Monarch-Titan-Scale-{NUM_NODES}_nodes-port_override",
        # machine=Machine.T4_X_4,  # Use GPU machines for training
        machine=machine_type,
        studio=studio,
        num_machines=num_nodes,
        env={
            "CUDA_VISIBLE_DEVICES": "0,1,2,3,4,5,6,7",  # Make all GPUs visible # TODO: Should make this one dynamic
            "MONARCH_FILE_LOG": "debug",
            "HYPERACTOR_REMOTE_ALLOC_ALLOWED_PORT_RANGE": "26601-26610",
            "HYPERACTOR_REMOTE_ALLOC_BIND_TO_INADDR_ANY": "true",
        },
    )

    print(f"Job started with ID: {job.name}")
    print(f"Job status: {job.status}")

    # Monitor job status
    return job, studio

In [9]:
# Launch the job
job, studio = launch_mmt_job(
    num_nodes=NUM_NODES, teamspace=TEAMSPACE, user=USER
)

print(f"Job launched. You can monitor it using: job.status")
print(f"To stop the job: job.stop()")
print(f"To clean up: studio.stop()")

Launching MMT job with 2 nodes...


INFO - Multi-Machine Job was successfully launched. View it at https://lightning.ai/meta-ai/general/jobs/Multi-Node-Monarch-Titan-Scale-2_nodes-port_override-hspx9?app_id=mmt


Job started with ID: Multi-Node-Monarch-Titan-Scale-2_nodes-port_override-hspx9
Job status: Pending
Job launched. You can monitor it using: job.status
To stop the job: job.stop()
To clean up: studio.stop()


In [10]:
from utils.master_node import run_master_server
cluster_info = run_master_server(expected_workers=NUM_NODES, port=HTTP_SERVER_PORT)

Master node IP: 3.84.102.51
Expecting 2 worker nodes to register...
Starting server on port 26600...
Waiting for workers... (0/2 registered) - Elapsed: 0s
Server started on 3.84.102.51:26600
Waiting for workers... (0/2 registered) - Elapsed: 30s
Waiting for workers... (0/2 registered) - Elapsed: 60s
Waiting for workers... (0/2 registered) - Elapsed: 90s
Waiting for workers... (0/2 registered) - Elapsed: 120s
Waiting for workers... (0/2 registered) - Elapsed: 150s
Waiting for workers... (0/2 registered) - Elapsed: 180s
Waiting for workers... (0/2 registered) - Elapsed: 210s
Waiting for workers... (0/2 registered) - Elapsed: 240s
Waiting for workers... (0/2 registered) - Elapsed: 270s
Waiting for workers... (0/2 registered) - Elapsed: 300s
Waiting for workers... (0/2 registered) - Elapsed: 330s
Waiting for workers... (0/2 registered) - Elapsed: 360s
Waiting for workers... (0/2 registered) - Elapsed: 390s
Waiting for workers... (0/2 registered) - Elapsed: 420s
Waiting for workers... (0/2 

52.14.215.20 - - [14/Oct/2025 00:41:36] "POST /register HTTP/1.1" 200 -


Registered worker node: 18.219.107.185 (2/2)
All worker nodes registered!
Registration server stopped
Final registered worker nodes: ['52.14.215.20', '18.219.107.185']
Worker IPs saved to /tmp/worker_nodes.txt
Cluster info saved to /tmp/cluster_info.json


18.219.107.185 - - [14/Oct/2025 00:41:40] "POST /register HTTP/1.1" 200 -


In [11]:
from utils.ip_utils import extract_ips_simple
worker_nodes_ip_file_path = "/tmp/worker_nodes.txt"
ip_addresses_set = extract_ips_simple(worker_nodes_ip_file_path)
ip_addresses_list = list(ip_addresses_set)
print(ip_addresses_list)

Extracted IP addresses:
18.219.107.185
52.14.215.20

IP set: {'18.219.107.185', '52.14.215.20'}
['18.219.107.185', '52.14.215.20']


In [ ]:
# ip_addresses_set = {'3.143.199.198', '3.132.52.102', '3.15.95.43'}
# ip_addresses_set = {'18.219.107.185', '52.14.215.20'}
tcp_addresses = [f"tcp!{ip}:{MONARCH_DEFAULT_PORT}" for ip in ip_addresses_set]

# # Or if you want to test it locally first on the local machine uncomment line below:
# tcp_addresses = ["tcp![::]:26600"]
# # For the local host machine only, please make sure that NUM_NODES is equal to 1;
# NUM_NODES = 1

print(*tcp_addresses)

tcp!18.219.107.185:26600 tcp!52.14.215.20:26600


# Example 1 - Run TorchTitan using Monarch for Llama 3 - 8B

In [4]:
from monarch._src.actor.allocator import RemoteAllocator, StaticRemoteAllocInitializer
from monarch._rust_bindings.monarch_hyperactor.alloc import AllocConstraints, AllocSpec
from monarch.actor import ProcMesh
import os

# os.environ["HYPERACTOR_REMOTE_PROCESS_ALLOC_PORT"] = "26600"
# os.environ["HYPERACTOR_REMOTE_PROCESS_ALLOC_ADDR"] = f"tcp!{public_master_host_ip_address}:{MONARCH_DEFAULT_PORT}"
# os.environ["HYPERACTOR_REMOTE_ALLOC_ALLOWED_PORT_RANGE"] = "26600-26610"
os.environ["HYPERACTOR_REMOTE_ALLOC_ALLOWED_PORT_RANGE"] = "26600-26610"
os.environ["HYPERACTOR_REMOTE_ALLOC_BOOTSTRAP_ADDR"] = f"tcp!{public_master_host_ip_address}:0"
# os.environ["HYPERACTOR_REMOTE_ALLOC_BOOTSTRAP_ADDR"] = "tcp!127.0.0.1:0"
# os.environ["HYPERACTOR_REMOTE_ALLOC_BOOTSTRAP_ADDR"] = "tcp!3.84.102.51:0"
# os.environ["HYPERACTOR_REMOTE_ALLOC_BOOTSTRAP_ADDR"] = "tcp!10.192.12.204:0"
os.environ["HYPERACTOR_REMOTE_ALLOC_BIND_TO_INADDR_ANY"] = "true"


allocator = RemoteAllocator(
        world_id="foo",
        initializer=StaticRemoteAllocInitializer(*tcp_addresses),
    )

alloc = allocator.allocate(
        AllocSpec(AllocConstraints(), hosts=NUM_NODES, gpus=NUM_GPUS)
    )

print(alloc)
# proc_mesh = await ProcMesh.from_alloc(alloc)
# proc_mesh = ProcMesh.from_alloc(alloc)

AllocHandle(_hy_alloc=<monarch._rust_bindings.monarch_hyperactor.pytokio.Shared object at 0x70ec091c9f20>, _extent={'hosts': 2, 'gpus': 8}, _stream_logs=True)


sys:1: UserWarning: The AllocSpec passed to RemoteAllocator.allocate has transport unix, but the transport from the remote process alloc initializer is tcp. This will soon be an error unless you explicitly configure monarch's default transport to tcp. The current default transport is unix.


In [5]:
proc_mesh = ProcMesh.from_alloc(alloc)

In [6]:
os.environ["HYPERACTOR_REMOTE_ALLOC_BOOTSTRAP_ADDR"]

'tcp!3.84.102.51:0'

In [7]:
import getpass
def get_job_name(num_hosts: int, num_gpus_per_host: int):
    return f"monarch-{getpass.getuser()}-hosts{num_hosts}-gpus{num_gpus_per_host}"
print(get_job_name(num_hosts=NUM_NODES, num_gpus_per_host=NUM_GPUS))

monarch-alisol-hosts2-gpus8


In [ ]:
import os
import sys
import logging
from monarch.actor import ProcMesh, Actor, endpoint, current_rank
import socket
from torchtitan.tools.logging import init_logger, logger
from torchtitan.train import Trainer
from typing import Optional
import torch
from torchtitan.config import JobConfig


class TitanTrainerWrapper(Actor):
    def __init__(self, job_config: JobConfig):
        self.rank = current_rank().rank
        self.job_config = job_config

    def _rprint(self, msg):
        """Helper method to print with rank information."""
        print(f"{self.rank=} {msg}")

    @endpoint
    def init(self):
        logging.getLogger().addHandler(logging.StreamHandler(sys.stderr))
        print(f"Initializing actor: {self.rank} {current_rank()=} {socket.gethostname()=}")


    @endpoint
    def train(self):
        logger.info("Starting training")
        config = self.job_config
        trainer: Optional[Trainer] = None

        try:
            trainer = Trainer(config)
            trainer.train()

            if config.checkpoint.create_seed_checkpoint:
                assert (
                    int(os.environ["WORLD_SIZE"]) == 1
                ), "Must create seed checkpoint using a single device, to disable sharding."
                assert (
                    # config.checkpoint.enable_checkpoint
                    config.checkpoint.enable
                ), "Must enable checkpointing when creating a seed checkpoint."
                trainer.checkpointer.save(curr_step=0, )
                logger.info("Created seed checkpoint")
            else:
                trainer.train()
        finally:
            if trainer:
                trainer.close()

            if torch.distributed.is_initialized():
                torch.distributed.destroy_process_group()
                logger.info("Process group destroyed.")
        print("Done training")

In [10]:
from torch.xpu import stream
from torchtitan.config import ConfigManager, JobConfig
from monarch.utils import setup_env_for_distributed

async def async_main(job_config: JobConfig):
    torch.use_deterministic_algorithms(True)
    job_name = get_job_name(NUM_NODES, NUM_GPUS)

    await setup_env_for_distributed(proc_mesh,
                                    )

    await proc_mesh.logging_option(stream_to_client=True, aggregate_window_sec=3)

    print(job_config)
    print(f"Spawning meshes on {job_name}")

    # trainer_actor = await proc_mesh.spawn("trainer_actor", TitanTrainerWrapper, job_config)
    trainer_actor = proc_mesh.spawn("trainer_actor", TitanTrainerWrapper, job_config)

    await trainer_actor.init.call()
    await trainer_actor.train.call()

In [11]:
init_logger()
config_manager = ConfigManager()

job_name = get_job_name(NUM_NODES, NUM_GPUS)

manual_args = [
        "--job.config_file",
        os.path.expanduser("/teamspace/studios/this_studio/torchtitan/torchtitan/models/llama3/train_configs/llama3_8b.toml"),
        "--model.tokenizer-path",
        # f"{FUSE_DST}/Llama-3.1-8B",
        "/teamspace/studios/this_studio/torchtitan/assets/hf/Llama-3.1-8B",
        "--training.steps",
        "25",
        "--training.dataset_path",
        # f"{FUSE_DST}/c4",
        "/teamspace/studios/this_studio/torchtitan/tests/assets/c4_test",
        "--job.dump_folder",
        # f"{FUSE_DST}/outputs/" + job_name,
        "/teamspace/studios/this_studio/torchtitan/outputs/" + job_name,
        "--training.seq_len",
        "1024",
        # "8192",
    ]
config = config_manager.parse_args(manual_args)
await async_main(config)

[titan] 2025-10-14 01:07:48,206 - root - WARNING - tokenizer_path is deprecated, use model.hf_assets_path instead. Setting hf_assets_path to tokenizer_path temporarily.
JobConfig(job=Job(config_file='/teamspace/studios/this_studio/torchtitan/torchtitan/models/llama3/train_configs/llama3_8b.toml', dump_folder='/teamspace/studios/this_studio/torchtitan/outputs/monarch-alisol-hosts2-gpus8', description='Llama 3 8B training', print_args=False), profiling=Profiling(enable_profiling=True, save_traces_folder='profile_trace', profile_freq=100, profiler_active=1, profiler_warmup=3, enable_memory_snapshot=False, save_memory_snapshot_folder='memory_snapshot'), metrics=Metrics(log_freq=1, enable_tensorboard=True, disable_color_printing=False, save_tb_folder='tb', save_for_all_ranks=False, enable_wandb=True), model=Model(name='llama3', flavor='8B', hf_assets_path='/teamspace/studios/this_studio/torchtitan/assets/hf/Llama-3.1-8B', tokenizer_path='/teamspace/studios/this_studio/torchtitan/assets/hf/L

>>> Aggregated Logs (2025-10-14 01:06:30) >>>
[16 similar log lines] Starting training
[16 similar log lines] Starting job: Llama 3 8B training
[5 similar log lines] [W1014 01:07:53.636464501 socket.cpp:767] [c10d] The client socket has failed to connect to [ip-10-192-12-106]:50173 (errno: 22 - Invalid argument).
[16 similar log lines] Building 1-D device mesh with ['dp_shard'], [16]
[16 similar log lines] [GC] Initial GC collection took 0.00 seconds
[11 similar log lines] Loading tokenizer from tokenizer.json
<<< Aggregated Logs (2025-10-14 01:07:55) <<<



>>> Aggregated Logs (2025-10-14 01:07:52) >>>
[15 similar log lines] Initializing actor: 9 current_rank()={'hosts': 1/2, 'gpus': 1/8} socket.gethostname()='ip-10-192-12-142'
<<< Aggregated Logs (2025-10-14 01:07:55) <<<



>>> Aggregated Logs (2025-10-14 01:07:55) >>>
[5 similar log lines] Loading tokenizer from tokenizer.json
[16 similar log lines] Preparing c4_test dataset from /teamspace/studios/this_studio/torchtitan/tests/assets/c4_test
[16 similar log lines] Building llama3 8B with TransformerModelArgs(_enforced='This field is used to enforce all fields have defaults.', dim=4096, n_layers=32, n_heads=32, n_kv_heads=8, vocab_size=128256, multiple_of=1024, ffn_dim_multiplier=1.3, norm_eps=1e-05, rope_theta=500000, rope_scaling_args=RoPEScalingArgs(scaling_factor=8.0, low_freq_factor=1.0, high_freq_factor=4.0, original_max_position_embeddings=8192), max_seq_len=1024, depth_init=True, use_flex_attn=False, attn_mask_type='causal', eos_id=0)
[16 similar log lines] CUDA capacity: NVIDIA L40S with 44.64GiB memory
[32 similar log lines] Peak flops undefined for: NVIDIA L40S, fallback to A100
[16 similar log lines] Model llama3 8B size: 8,030,261,248 total parameters
[16 similar log lines] Applied selective 

>>> Aggregated Logs (2025-10-14 01:07:55) >>>
[14 similar log lines] Done training
<<< Aggregated Logs (2025-10-14 01:15:14) <<<



>>> Aggregated Logs (2025-10-14 01:15:11) >>>
[15 similar log lines] step: 25  loss:  8.3818  grad_norm:  3.8415  memory: 18.47GiB(41.37%)  tps: 59  tflops: 2.76  mfu: 0.88%
[31 similar log lines] Training completed
[16 similar log lines] Training starts at step 26
[2 similar log lines] Sleeping 2 seconds for other ranks to complete
[16 similar log lines] Profiling active. Traces will be saved at /teamspace/studios/this_studio/torchtitan/outputs/monarch-alisol-hosts2-gpus8/profile_trace
[14 similar log lines] Process group destroyed.
<<< Aggregated Logs (2025-10-14 01:15:14) <<<

>>> Aggregated Logs (

>>> Aggregated Logs (2025-10-14 01:15:14) >>>
[2 similar log lines] Done training
<<< Aggregated Logs (2025-10-14 01:15:17) <<<



2025-10-14 01:15:14) >>>
[1 similar log lines] Training completed
[1 similar log lines] wandb: updating run metadata
[3 similar log lines] wandb: 
[1 similar log lines] wandb: Run history:
[2 similar log lines] wandb:                    grad_norm ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁
[1 similar log lines] wandb: loss_metrics/global_avg_loss ▇▆▅▆▆▇▆▅▄▄▃▃▂▂▂▂▁▁▁█▂▁▁▁▁
[1 similar log lines] wandb: loss_metrics/global_max_loss ▅▄▄▅▄▆▅▄▃▃▂▄▃▂▂▁▂▁▂█▂▂▂▁▂
[1 similar log lines] wandb:                           lr ▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇▇██
[4 similar log lines] wandb:         memory/max_active(%) ▁████████████████████████
[1 similar log lines] wandb:     memory/num_alloc_retries ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
[2 similar log lines] wandb:                           +7 ...
[1 similar log lines] wandb: Run summary:
[2 similar log lines] Process group destroyed.
[1 similar log lines] wandb:                    grad_norm 3.84154
[1 similar log lines] wandb: loss_metrics/global_avg_loss 8.38181
[1 similar log lines] wa

In [ ]:
proc_mesh

In [ ]:
dir(TitanTrainerWrapper)

In [ ]:
await proc_mesh.stop()